In [4]:
import math
import cv2
import time
import numpy as np
from math import hypot
import import_ipynb
import HandTrackingModule as htm  # Custom module for hand tracking
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Webcam parameters
wCam, hCam = 640, 360

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
cap.set(cv2.CAP_PROP_FPS, 30)  # Limit to 30 FPS for smoother performance
pTime = 0

# Initialize hand detector
detector = htm.handDetector(detectionCon=0.7)

# Initialize audio utilities
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]

volBr = 400  # Visual bar height
volPer = 0   # Volume percentage

# Main loop
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flip the image for a mirror effect
    img = detector.findHands(img)  # Detect hands
    lmList = detector.findPosition(img, draw=False)

    if len(lmList) != 0:
        # Extract coordinates for thumb and index finger tips
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        # Draw landmarks and line
        cv2.circle(img, (x1, y1), 7, (255, 80, 100), cv2.FILLED)
        cv2.circle(img, (x2, y2), 7, (255, 80, 100), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 80, 100), 3)
        cv2.circle(img, (cx, cy), 7, (255, 80, 100), cv2.FILLED)

        # Calculate the length of the line
        length = math.hypot(x2 - x1, y2 - y1)

        # Map the length to volume and bar values
        vol = np.interp(length, [50, 250], [minVol, maxVol])
        volBr = np.interp(length, [50, 250], [400, 150])
        volPer = np.interp(length, [50, 250], [0, 100])

        # Set the system volume
        volume.SetMasterVolumeLevel(vol, None)

        # Draw a different color circle if the length is below 50
        if length < 50:
            cv2.circle(img, (cx, cy), 7, (0, 80, 100), cv2.FILLED)

    # Draw the volume bar
    cv2.rectangle(img, (50, 150), (85, 400), (255, 80, 100), 3)
    cv2.rectangle(img, (50, int(volBr)), (85, 400), (255, 80, 100), cv2.FILLED)

    # Display the volume percentage
    cv2.putText(img, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_PLAIN, 3,
                (255, 0, 0), 3)

    # Calculate and display FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv2.putText(img, f'FPS : {int(fps)}', (40, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                (255, 0, 0), 3)

    cv2.imshow("Img", img)

    # Quit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
